In [1]:
from os import path,listdir
from datetime import datetime,timedelta
import time
import requests
from stem import Signal
from stem.control import Controller
import re
import pandas as pd
import numpy as np
import random
import pytz
from tqdm import tqdm
local_tz = 'Asia/Almaty'

In [ ]:
curtime=pytz.timezone(local_tz).localize(datetime.strptime('2000-01-01 '+'23:55', '%Y-%m-%d %H:%M'))
curtime

In [ ]:
datetime(2020, 10, 17, 1, 0).timestamp()

In [ ]:
datetime.fromtimestamp(1604170800)

In [ ]:
def getone(pattern, string):
    m = pattern.search(string)
    res = m.group(1) if m else ''
    return res.strip()

def clean(txt):
    return txt.replace('&','').replace('\'','').strip()

def get_cookie_value(name):
    return [x['value'] for x in cookies if x['name']==name][0]

In [ ]:
base_url='https://sofascore.com/'
api_url='https://api.sofascore.com/api/v1/'

def generate_headers():
    return {
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-US,en;q=0.5",
            "Connection": "keep-alive",
            "Host": "api.sofascore.com",
            "Origin": "https://www.sofascore.com",
            "Referer": referer,
            "TE": "Trailers",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0"
        }

def load_match_info(did, isft):
    load_json('lineups',did, headers)
    load_json('votes',did, headers, isft=isft)
    if isft==1:
        load_json('incidents',did, headers)
        load_json('graph',did, headers)
    #load_json('best-players',did, headers)
    #load_json('event',did, headers)
    #load_json('statistics',did, headers)
    #load_json('h2h',did, headers)
    #load_json('pregame-form',did, headers)
    #load_json('winning-odds',did, headers)

#

def load_json(fn, did, headers, isft=0):
    file_name='raw/{}_{}_{:%Y-%m-%d-%H%M}.json'.format(fn, did, datetime.now()) if fn=='votes' else f'raw/{fn}_{did}.json'
    if not path.exists(file_name) or (fn=='votes' and isft==0):
        script='' if fn=='event' else '/provider/1/'+fn if fn=='winning-odds' else '/'+fn
        link=f'{api_url}event/{did}{script}'
        r = requests.get(link, headers=headers)
        if r.status_code==200:
            with open(file_name, 'w+', encoding='utf8') as f:
                f.write(r.text)



In [9]:
file_name='data/matches.csv'
df_matches=pd.read_csv(file_name, index_col=None)
#SERVER_TZ = 'UTC'
#df_matches['ts']=pd.DatetimeIndex(pd.to_datetime(df_matches['startTimestamp'], unit='s')).tz_localize(SERVER_TZ)
df_matches=df_matches.drop_duplicates(keep='first')

In [10]:
df_matches_done=df_matches.loc[df_matches['done']==1]
df_matches=df_matches.loc[df_matches['done']<1]
df_matches_done.to_csv('data/matches_done.csv', index=False)
df_matches.to_csv('data/matches1.csv', index=False)


In [8]:
df_matches

,homeTeamShort,awayTeam,tournament,awayTeamShort,homeTeam,startTimestamp,id,awayScoreFT,awayScoreET,done,coverage,homeScoreFT,winnerCode,homeScoreHT,round,country,status,ts,homeScoreET,awayScoreHT
108,Ipswich,Derby County,championship,Derby,Ipswich Town,1420892100,5583876,1.0,1.0,0,1,0.0,2,0.0,25.0,england,100,2015-01-10 12:15:00+00:00,0.0,0.0
110,Troyes,Stade Brestois 29,ligue-2,Brest,Troyes,1420894800,5510536,0.0,0.0,0,1,1.0,1,1.0,19.0,france,100,2015-01-10 13:00:00+00:00,1.0,0.0
111,Veria,AEL Kalloni,super-league,Kalloni,Veria NFC,1420894800,6570345,1.0,1.0,0,1,1.0,3,1.0,11.0,greece,100,2015-01-10 13:00:00+00:00,1.0,1.0
112,Burnley,Queens Park Rangers,premier-league,QPR,Burnley,1420902000,5582834,1.0,1.0,0,1,2.0,1,2.0,21.0,england,100,2015-01-10 15:00:00+00:00,2.0,1.0
113,Chelsea,Newcastle United,premier-league,Newcastle,Chelsea,1420902000,5582836,0.0,0.0,0,1,2.0,1,1.0,21.0,england,100,2015-01-10 15:00:00+00:00,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39988,Chaves,Portimonense,primeira-liga,Portimonense,GD Chaves,1524409200,7473301,1.0,1.0,0,1,2.0,1,0.0,31.0,portugal,100,2018-04-22 15:00:00+00:00,2.0,0.0
39990,O'Higgins,Curicó Unido,primera-division,Curicó Unido,O'Higgins,1524409200,7712202,2.0,2.0,0,1,2.0,3,0.0,10.0,chile,100,2018-04-22 15:00:00+00:00,2.0,0.0
39993,Santa Clara,FC Famalicão,segunda-liga,Famalicão,CD Santa Clara,1524409200,7516778,0.0,0.0,0,3,2.0,1,2.0,35.0,portugal,100,2018-04-22 15:00:00+00:00,2.0,0.0
39994,Leixões,Arouca,segunda-liga,Arouca,Leixões SC,1524409200,7516783,2.0,2.0,0,3,2.0,3,1.0,35.0,portugal,100,2018-04-22 15:00:00+00:00,2.0,2.0


In [ ]:
exclude=['india', 'peru', 'south-africa', 'germany-amateur', 'saudi-arabia', 'united-arab-emirates']
df_matches=df_matches.loc[~df_matches['country'].isin(exclude)]

In [ ]:
df_matches.to_csv('data/matches1.csv', index=False)

In [ ]:
df_matches_new=pd.read_csv('data/matches1.csv', index_col=None).set_index('id')
df_matches=pd.read_csv('data/matches.csv', index_comatches


In [ ]:
df_matches.update(df_matches_new[['awayScoreHT','awayScoreET','homeScoreET','awayScoreFT','homeScoreFT','status','winnerCode','homeScoreHT']])
df_matches['id'] = df_matches.index
df_matches_new['id'] = df_matches_new.index
matchespd.concat([df_matches,df_matches_new]).drop_duplicates(subset='id', keep='first').reset_index(drop=True)

In [ ]:
LOCAL_TZ = 'Asia/Almaty'
df_matches['ts'].apply(lambda d: d.astimezone(pytz.timezone(LOCAL_TZ)))

In [ ]:
avoid_countries=['saudi-arabia', 'chile']
PROXY = "127.0.0.1:9051"

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options



webdriver.DesiredCapabilities.FIREFOX['proxy'] = {
    "socksProxy": PROXY,
    "socksVersion": 5,
    "proxyType": "MANUAL",
}

options = webdriver.ChromeOptions()
options.add_argument('--proxy-server=socks5://' + PROXY)
#chrome = webdriver.Chrome(executable_path=r'../lib/chromedriver.exe', options=options)
firefox = webdriver.Firefox(executable_path=r'../lib/geckodriver.exe')

firefox.get("https://www.sofascore.com/")
#chrome.get("https://www.sofascore.com/")

referer=firefox.current_url
cookies=firefox.get_cookies()
html = firefox.page_source


In [ ]:
html = firefox.page_source

dids=[]

keys=['svg','path','rect','span']
for key in keys:
    html=re.sub(f'<{key}[^>]+>','',html)
    html=html.replace(f'</{key}>','')
html=re.sub(r'<div data-change-key="status"[^>]+>-</div>','',html)
html=html.replace(f'</div>','')

pChamp = re.compile(r'<a href="/tournament/football/([^/]+)/([^/]+)/(\d+)">([^<]+)</a>')
pMatch = re.compile(r'<a data-id="(\d+)" class="[^"]+" href="([^"]+)"><div><div[^>]+><div[^>]+><div[^>]+>([^<]+)<div[^>]+>([^<]+)<div[^>]+><div[^>]+><div[^>]+>([^<]+)<div[^>]+>([^<]+)<div[^>]+><div[^>]+><div[^>]+><div[^>]+>([^<]*)<div[^>]+>([^<]*)')
pMatch1 = re.compile(r'<a data-id="(\d+)" class="[^"]+" href="([^"]+)"><div><div[^>]+><div title="([^"]+)"[^>]+><div[^>]+>(<div[^>]+>)?([^<]+)')
pMatch2 = re.compile(r'[^>]+><div[^>]+>([^<]+)<div[^>]+>([^<]+)<div[^>]+><div[^>]+><div[^>]+><div[^>]+>([^<]*)<div[^>]+>([^<]*)')
parts = html.split('<div class="Section-sc-1a7xrsb-0 styles__CategoryImageSection-sc-7mecl8-8 hTnyYh">')[1:-1]
for part in parts:
    if 'SofaScore ratings' in part:
        m = pChamp.search(part)
        if m:
            country, champ_sname, champ_id, champ_name=m.groups()
        else:
            raise Exception('CHAMPS NOT FOUND IN\n\n'+part)
        if country in avoid_countries:
            continue
        mm=part.split('</button></a>')[:-1]
        print(country, champ_sname, champ_id, champ_name,'FOUND %s matches'%(len(mm)))
        for match in mm:
            if 'Canceled' in match or 'Postponed' in match:
                print('Canceled or Postponed!!!')
                continue

            m = pMatch1.search(match)
            if m:
                match_did,match_link,match_status,_,match_time=m.groups()
                print(match_did,match_link,match_status,match_time)
                if match_time=='HT':
                    continue
            else:
                raise Exception('MATCH NOT FOUND IN\n\n'+match)
            pps=match.split('<div title=')[-1]
            m = pMatch2.search(pps)
            if m:
                match_team1,match_team2,match_score1,match_score2=m.groups()
                print(match_team1,match_team2,match_score1,match_score2)
            else:
                raise Exception('MATCH NOT FOUND IN PPS\n\n'+pps)
            dids.append([country, champ_sname, champ_id, champ_name,match_did,match_link,match_time,match_status,match_team1,match_team2,match_score1,match_score2])
matches=pd.DataFrame(data=dids, columns=['country', 'champ_sname', 'champ_id', 'champ_name','match_did','match_link','match_time','match_status','match_team1','match_team2','match_score1','match_score2'])
matches.loc[matches['match_score1']=='','match_score1']=np.nan
matches.loc[matches['match_score2']=='','match_score2']=np.nan
matches['FT']=np.where(matches['match_score1'].isna(),0,1)
dids=matches[['match_did','FT']].values
np.random.shuffle(dids)

In [ ]:
d=datetime.strptime('2020-10-31', '%Y-%m-%d')
dstr='{:%Y-%m-%d}'.format(d)
matches['ts']=matches['match_time'].apply(lambda x: pytz.timezone(local_tz).localize(datetime.strptime(f'{dstr} {x}', '%Y-%m-%d %H:%M')))
matches.to_csv(f'raw/matches_{dstr}.csv')

In [ ]:
def load_matches(arr):
    l=len(arr)
    if l>0:
        did,isft=arr[0]
        print(f'Loading {did},{isft} from {l}...', end='')
        load_match_info(did, isft)
        print(' done!')
        time.sleep(random.uniform(1, 5))
        load_matches(np.delete(arr, 0, 0))

In [ ]:
load_match_info(1399303, 1)

In [ ]:
headers=generate_headers()
load_matches(dids)

In [ ]:
dids

In [ ]:
fn='statistics'
did=9091949
script='' if fn=='event' else '/'+fn
link=f'{api_url}event/{did}{script}'
headers=generate_headers()
r = requests.get(link, headers=headers)
if r.status_code==200:
    with open(f'raw/{fn}_{did}.json', 'w+', encoding='utf8') as f:
        f.write(r.text)

In [ ]:
proxies = {'https': f'socks5://{PROXY}'}

dates=[]
d= datetime(2013, 1, 1)
end_date= datetime(2020, 10, 28)

while d<=end_date:
    dates.append(d)
    d+=timedelta(days=1)
dates=np.array(dates) 
np.random.shuffle(dates)

In [ ]:
def tor_new_identity():
    with Controller.from_port(port=9052) as controller:
        controller.authenticate()
        #controller.authenticate(password="1234")
        controller.signal(Signal.NEWNYM)
        controller.close()

def load_days(test=False,c=0,n=0,max_load=random.randint(50, 100)):
    global dates,r,dstr,proxies
    l=len(dates)
    if l>0:
        dstr='{:%Y-%m-%d}'.format(dates[0])
        link=f'https://api.sofascore.com/api/v1/sport/football/scheduled-events/{dstr}'
        referer=f'https://www.sofascore.com/football/{dstr}'
        print(f'Loading {dstr} from {l}...', end='')
        headers=generate_headers()
        r = requests.get(link, headers=headers, proxies=proxies)
        if test:
            print(r.status_code)
        if r.status_code==200:
            c=0
            with open(f'raw/{dstr}.json', 'w+', encoding='utf8') as f:
                f.write(r.text)
            n+=1
            if n>max_load:
                tor_new_identity()
                time.sleep(random.uniform(10, 20))
                load_days()
        else:
            print(f'ERROR {r.status_code}!!! Obtaining new identity. Try # {c}.')
            tor_new_identity()
            time.sleep(random.uniform(10, 20))
            c+=1
            if c>10:
                return 
            load_days(c=c)
        c=0
        print(f' done #{n} from the batch of {max_load}!')
        time.sleep(random.uniform(1, 5))
        dates=np.delete(dates, 0, 0)
        if not test:
            load_days(c=c,n=n,max_load=max_load)

In [ ]:
load_days(test=False)
#len(dates)

In [ ]:
fn='winning-odds'
did=9091949
script='' if fn=='event' else '/provider/1/'+fn if fn=='winning-odds' else '/'+fn
link=f'{api_url}event/{did}{script}'
link

In [ ]:
from data_provider import DataProvider
ds='2014-01-01'
de='2020-10-31'

#ds='2020-04-05'
#de='2020-04-05'

dp=DataProvider()
#dp.load_days(ds, de)
#dp.load_days() # Process al files
dp.load_matches()

In [ ]:
#matches=[x for x in dp.DATA['events'] if 'coverage' in x.keys()]
dp.DATA

In [ ]:
[datetime.strptime(f.replace('.json', ''), '%Y-%m-%d') for f in listdir('raw/')] 

In [ ]:
file_name='data/matches1.csv'
dp.df_matches.to_csv(file_name, index=False)

In [ ]:
dp.COUNTER

In [ ]:
import json
file_name='raw/2016-08-28.json'
with open(file_name, 'r', encoding='utf8') as f:
    data=json.load(f)
matches=[x for x in data['events'] if 'coverage' in x.keys()]
matches=[x for x in matches if x['coverage']>-1]
matches=[x for x in matches if not x['status']['code'] in [60,70]]
len(matches)

In [ ]:
[x for x in matches if not x['status']['code']==100 and not x['status']['code']==110 and not x['status']['code']==0]

In [ ]:
matches=[x for x in dp.data['events'] if 'coverage' in x.keys()]
matches=[x for x in matches if x['coverage']>-1]
matches=[x for x in matches if x['status']['code']==100 or x['status']['code']==110]
len(matches)

In [ ]:
a=[  {'tournament' : x['tournament']['slug'],
    'country' : x['tournament']['category']['slug'],
    'round' : x['roundInfo']['round'] if 'roundInfo' in x.keys() else np.NaN,
    'status' : x['status']['code'],
    'homeTeam' : x['homeTeam']['name'],
    'homeTeamShort' : x['homeTeam']['shortName'],
    'awayTeam' : x['awayTeam']['name'],
    'awayTeamShort' : x['awayTeam']['shortName'],
    'homeScoreFT' : x['homeScore']['normaltime'] if 'normaltime' in x.keys() else np.NaN,
    'homeScoreET' : x['homeScore']['current'] if 'current' in x.keys() else np.NaN,
    'homeScoreHT' : x['homeScore']['period1'] if 'period1' in x.keys() else np.NaN,
    'awayScoreFT' : x['awayScore']['normaltime'] if 'normaltime' in x.keys() else np.NaN,
    'awayScoreET' : x['awayScore']['current'] if 'current' in x.keys() else np.NaN,
    'awayScoreHT' : x['awayScore']['period1'] if 'period1' in x.keys() else np.NaN,
    'id' : x['id'],
    'startTimestamp' : x['startTimestamp'],
    'coverage' : x['coverage'],
    'winnerCode' : x['winnerCode']} for x in matches]

In [ ]:
pd.DataFrame(data=a)


In [ ]:
load_json('best-players',did, headers)

In [ ]:
did=9091942
load_match_info(did)